In [1]:
from tensorboardX import SummaryWriter
from validate import validate
from networks.trainer import Trainer
from torch.utils.data import DataLoader
import numpy as np
import os
import time
import random
import torch

#from base_miner.util import Logger
from util.data import load_datasets, create_real_fake_datasets
from options import TrainOptions

/home/user/mambaforge/envs/bmsn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'Gustavosta/MagicPrompt-Stable-Diffusion': {'model': 'Gustavosta/MagicPrompt-Stable-Diffusion', 'tokenizer': 'gpt2', 'device': -1}}
{'stabilityai/stable-diffusion-xl-base-1.0': {'use_safetensors': True, 'variant': 'fp16'}, 'SG161222/RealVisXL_V4.0': {'use_safetensors': True, 'variant': 'fp16'}, 'Corcelio/mobius': {'use_safetensors': True}}


In [2]:
opt = TrainOptions().parse()

model = Trainer(opt)
weight_path = '../mining_models/model_epoch_best.pth'
model.model.load_state_dict(torch.load(weight_path, map_location='cpu'))
model.model.eval();

----------------- Options ---------------
                     arch: res50                         
               batch_size: 64                            
                    beta1: 0.9                           
                blur_prob: 0                             
                 blur_sig: 0.5                           
          checkpoints_dir: ./checkpoints                 
                class_bal: False                         
                  classes:                               
           continue_train: False                         
                 cropSize: 224                           
                 data_aug: False                         
                 dataroot: ./dataset/                    
                delr_freq: 20                            
          earlystop_epoch: 15                            
                    epoch: latest                        
              epoch_count: 1                             
                  gpu_ids: 0  

In [4]:
from bitmind.constants import DATASET_META

real_datasets, fake_datasets = load_datasets(DATASET_META)
train_dataset, val_dataset, test_dataset = create_real_fake_datasets(real_datasets, fake_datasets)


[{'path': 'poloclub/diffusiondb', 'name': 'large_first_50k', 'create_splits': True}]
Loading poloclub/diffusiondb[train] ... 

/home/user/mambaforge/envs/bmsn/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for poloclub/diffusiondb contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/poloclub/diffusiondb
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


done, len=40000
Loading poloclub/diffusiondb[validation] ... done, len=5000
Loading poloclub/diffusiondb[test] ... done, len=5000
Loading dalle-mini/open-images[train] ... done, len=9011219
Loading merkol/ffhq-256[train] ... done, len=56000
Loading jlbaker361/flickr_humans_20k[train] ... done, len=16000
Loading saitsharipov/CelebA-HQ[train] ... done, len=162079
Loading dalle-mini/open-images[validation] ... done, len=41620
Loading merkol/ffhq-256[validation] ... done, len=7000
Loading jlbaker361/flickr_humans_20k[validation] ... done, len=2000
Loading saitsharipov/CelebA-HQ[validation] ... done, len=20260
Loading dalle-mini/open-images[test] ... done, len=125436
Loading merkol/ffhq-256[test] ... done, len=7000
Loading jlbaker361/flickr_humans_20k[test] ... done, len=2000
Loading saitsharipov/CelebA-HQ[test] ... done, len=20260


In [ ]:
from bitmind.real_fake_dataset import RealFakeDataset
from bitmind.image_dataset import ImageDataset
from bitmind.image_transforms import base_transforms

bm_ds = ImageDataset('bitmind/RealVisXL_V4.0_images', 'train', None, True, None)
ds = RealFakeDataset([bm_ds], [bm_ds], fake_prob=0, transforms=base_transforms)
test_loader = DataLoader(
    ds, batch_size=32, shuffle=False, num_workers=0, collate_fn=lambda d: tuple(d))



In [5]:
train_loader = DataLoader(
    train_dataset, batch_size=32, shuffle=True, num_workers=0, collate_fn=lambda d: tuple(d))
val_loader = DataLoader(
    val_dataset, batch_size=32, shuffle=False, num_workers=0, collate_fn=lambda d: tuple(d))
test_loader = DataLoader(
    test_dataset, batch_size=32, shuffle=False, num_workers=0, collate_fn=lambda d: tuple(d))



In [6]:
model.eval()
acc, ap, r_acc, f_acc, y_true, y_pred = validate(model.model, test_loader)
print("(Test) acc: {}; ap: {}".format(acc, ap))

(Test) acc: 0.9555; ap: 0.9920430338580721


In [ ]:
from sklearn.metrics import average_precision_score, precision_recall_curve, accuracy_score
import matplotlib.pyplot as plt

pred_idx = 0
test_loader.dataset.reset()
with torch.no_grad():
    y_true, y_pred = [], []
    for batch in test_loader:
        keep_idx = [i for i, b in enumerate(batch) if b[0].shape[0] == batch[0][0].shape[0]]
        # batch = np.array(batch)
        inputs = torch.stack([b[0] for i, b in enumerate(batch) if i in keep_idx])
        labels = torch.stack([torch.tensor(b[1]) for i, b in enumerate(batch) if i in keep_idx])
        img, label = inputs.to('cuda').float(), labels.to('cuda').float()

        out = model.model(img).sigmoid().flatten().tolist()
        y_pred.extend(out)
        y_true.extend(label.flatten().tolist())

        for y, y_hat, img, idx in zip(label.flatten().tolist(), out, img, keep_idx):
            if y != np.round(y_hat):
                plt.imshow(img.cpu().permute(1,2,0).numpy())
                plt.show()
                img_id = test_loader.dataset._history['index'][pred_idx + idx]
                img_source = test_loader.dataset._history['source'][pred_idx + idx]
                print(f"source: {img_source}; id: {img_id}; Label: {y}; Prediction: {y_hat}")
        pred_idx += len(keep_idx)

y_true, y_pred = np.array(y_true), np.array(y_pred)
r_acc = accuracy_score(y_true[y_true==0], y_pred[y_true==0] > 0.5)
f_acc = accuracy_score(y_true[y_true==1], y_pred[y_true==1] > 0.5)
acc = accuracy_score(y_true, y_pred > 0.5)
ap = average_precision_score(y_true, y_pred)


In [ ]:
def compute_metrics(TP, FP, TN, FN):
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    if (precision + recall) == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)
    
    return precision, recall, f1_score

In [ ]:
tp = sum(y_pred[y_true==1] > 0.5)
fp = sum(y_pred[y_true==0] > 0.5)
tn = sum(y_pred[y_true==0] <= 0.5)
fn = sum(y_pred[y_true==1] <= 0.5)
tp, fp, tn, fn

In [ ]:
compute_metrics(tp, fp, tn, fn)

In [ ]:
len(test_loader), len(test_dataset)